In [79]:
#импортируем необходимые библиотеки 
import pandas as pd #для обработки и представления данных 
import nltk #обработка слов и ведение их статистики
import numpy as np #манипуляции с матрицами 
from scipy.sparse import hstack #склейка матриц
from sklearn.feature_extraction.text import TfidfVectorizer #кодирование текстовой информации
from sklearn.model_selection import KFold, cross_val_score #перекретсная проверка
from sklearn.linear_model import LogisticRegression #логистическая регрессия 
from sklearn.metrics import accuracy_score #метрика качества 
from sklearn.model_selection import train_test_split #разбиение выборки
import lightgbm as lgb #эффективный алгоритм грудиентного бустинга

In [65]:
#считывание данных в dataframe
data = pd.read_csv("train.csv")
categories = pd.read_csv("category.csv")
data.head()

,item_id,title,description,price,category_id
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1


In [3]:
#вывод описания первых 20 объявлений для примерного представления
data["description"][:20]

0                            Гобелен. Размеры 139х84см.
1     Продам недорого 4 стула из светлой прессованно...
2     Мини баня МБ-1(мини сауна), предназначена для ...
3     Продам эксклюзивную коллекцию книг, выпущенную...
4     Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...
5             Состояние хорошее. Имеется теплый чехол .
6     Смесь молочная адаптированная ультрапастеризов...
7     Часы абсолютно новые! с коробкой. Часы   Китай...
8     Санатории Белоруссии! -  «Лепельский военный» ...
9     Фотохолст на подрамнике. 36х58см. Галерейная н...
10        В хорошем состоянии,тёмно-коричневый цвет.Юмр
11    Игрушка playGro, б/у, состояние отличное. (Вид...
12    Кроватка для новорожденного регулируется в 2 п...
13    Продам утяжелители поясные новые, 2 кг. 400р. ...
14                   продам люстру,в отличном состоянии
15    Семена фацелии-отличный медонос,нетребовательн...
16    В хорошем состоянии . С полочками .105 на 50 С...
17    Продам деревообрабатывающий станок . На ин

In [4]:
#функция для обработки текста, все, что не является буквами русского или латинского алфавита или числами, заменяется на пробел
def text_transform(text: pd.Series) -> pd.Series:
    return text.str.lower().replace("[^а-яА-Яa-zA-Z0-9]", " ", regex=True)

In [24]:
#здесь в векторайзер на н-граммах(для небольшой оценки контекста), max_df убирает самые популярные слова, 
#встречающиеся в 95 процентах объявлений, min_df наоборот убирает редкие, stop_words также наиболее часто используемые 
#vectorizer_with_bounds = TfidfVectorizer(ngram_range = (1, 2), max_df=0.95, min_df=0.05, stop_words=["english","russian"])
#X_train_text_descr = vectorizer_with_bounds.fit_transform(text_transform(data["description"]))
#X_train_text_title = vectorizer2.fit_transform(text_transform(data["title"]))

In [66]:
#здесь почти такое же преобразование слов в признаки, кроме min_df. В данном задании оказалось лучше не убирать редкие слова 
vect1 = TfidfVectorizer(ngram_range = (1, 2), max_df=0.95, stop_words=["english","russian"]) 
X_train_text_descr = vect1.fit_transform(text_transform(data["description"])) #преобразование описаний(кодирование слов)
vect2 = TfidfVectorizer(ngram_range = (1, 2), max_df=0.95, stop_words=["english","russian"])
X_train_text_title = vect2.fit_transform(text_transform(data["title"])) #преобразование названий объявлений 

In [67]:
X_train = hstack([X_train_text_descr, X_train_text_title]) #соединение по горизонтали двух матриц слов
X_train_price = data["price"].values.reshape(-1,1) #извлечение цен в матрицу и переворот в вертикаль
X_train_price

array([[ 1000.],
       [ 1250.],
       [13000.],
       ...,
       [ 2900.],
       [  300.],
       [19000.]])

In [68]:
X_train = hstack([X_train, X_train_price]) #финальное соединение матриц слов и массива цен в общую, готовую для обучения,выборку

In [69]:
y_train = data["category_id"] #извлечение категорий товаров

In [70]:
#разбиение выборки на тренировочную и тестовую в пропорциях по умолчанию(0.75 тренировочная 0.25 тестовая) с перемешиванием
X_tr, X_ts, y_tr, y_ts = train_test_split(X_train,y_train, random_state = 42, shuffle=True)

In [72]:
lgbm = lgb.LGBMClassifier() #создание объекта классификатора 
lgbm.fit(X_tr, y_tr) #обучение модели
y_pr = lgbm.predict(X_ts) #предсказывание моделью на тестовой выборке
print("%.2f" % accuracy_score(y_ts, y_pr)) #вычисление качества метрикой accuracy
                                           #сравнение предсказанных моделью и правильных ответов

0.85


In [12]:
data_test = pd.read_csv("test.csv") #считывание csv файла в новый dataframe
data_test.head()

,item_id,title,description,price
0,489517,Стоик журнальный сталь,продам журнальный столик изготавливаю столы из...,10000.0
1,489518,iPhone 5 64Gb,"Телефон в хорошем состоянии. Комплект, гаранти...",12500.0
2,489519,Утеплитель,ТЕПЛОПЕЛЕН-ЛИДЕР ТЕПЛА!!! Толщина утеплителя :...,250.0
3,489520,Пальто демисезонное,Продам пальто женское (букле) в отличном состо...,1700.0
4,489521,Samsung syncmaster T200N,"Условно рабочий, проблема в панели настройки м...",1000.0


In [13]:
#Такая же процедура обработки тестовых данных, отличие только в используемом методе в кодировании текста
#на тестовой выборке используется transform, а не fit_transform 
X_test_text_descr = vect1.transform(text_transform(data_test["description"]))
X_test_text_title = vect2.transform(text_transform(data_test["title"]))
X_test = hstack([X_test_text_descr, X_test_text_title])
X_test_price = data_test["price"].values.reshape(-1,1)
X_test = hstack([X_test, X_test_price])

In [14]:
y_pred = lgbm.predict(X_test) #предсказание моделью овтетов на тестовой выборке

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:478: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [54]:
X_test_id = data_test["item_id"].values.reshape(-1,1) #извлечение id объявлений и переворот в вертикаль
predicted = y_pred.reshape(-1,1)
final = np.hstack((X_test_id, predicted)) #сопоставление id объявления и полученных ответов
final

array([[489517,     22],
       [489518,      0],
       [489519,     15],
       ...,
       [732680,      7],
       [732681,     18],
       [732682,     29]], dtype=int64)

In [52]:
np.savetxt("test_.csv", final, delimiter=",", fmt="%d") #сохранение овтетов классификатора в файл

In [73]:
#на данном этапе просчитывается качество на рзных уровнях иерархии 
#здесь товары разбиты на первые, наиболее общие категории (бытовая электроника, Для дома и дачи, Личные вещи, Хобби и отдых)
y_train1 = data["category_id"] #извлечение целевой переменной и последующая замена значений
y_train1 = np.where(y_train1 < 15, 0, y_train1)  #все, что бытовая электроника заменяется на 0(там, где id категории 0-14)
y_train1 = np.where((y_train1 > 14) & (y_train1 < 30), 1, y_train1) #все, что Для дома и дачи заменяется на 1
y_train1 = np.where((y_train1 > 29) & (y_train1 < 42), 2, y_train1)#все, что Личные вещи заменяется на 2
y_train1 = np.where(y_train1 > 41, 3, y_train1) #замена значений все, что Хобби и отдых заменяется на 3

In [74]:
X_tr1, X_ts1, y_tr1, y_ts1 = train_test_split(X_train, y_train1, random_state = 42, shuffle=True) #снова разбиение
lgbm1 = lgb.LGBMClassifier() #создание объекта классификатора 
lgbm1.fit(X_tr1, y_tr1) #обучение модели
y_pr1 = lgbm1.predict(X_ts1) #предсказывание моделью на тестовой выборке
print("%.2f" % accuracy_score(y_ts1, y_pr1)) #вычисление качества метрикой accuracy
                                            #сравнение предсказанных моделью и правильных ответов

0.88


In [75]:
#здесь товары разбиты на следующем уровне иерархии (Телефоны, Ноутбуки, .., Ремонт и строительство, .., Музыкальные инструменты)
y_train2 = data["category_id"]
y_train2 = np.where((y_train2 == 0) | (y_train2 == 2) | (y_train2 == 6) | ((y_train2 >= 9) & (y_train2 <=11)) , 0, y_train2)
y_train2 = np.where(y_train2 == 1, 1, y_train2)
y_train2 = np.where(y_train2 == 3, 2, y_train2)
y_train2 = np.where((y_train2 == 4) | (y_train2 == 8), 3, y_train2)
y_train2 = np.where((y_train2 == 5) | (y_train2 == 12), 4, y_train2)
y_train2 = np.where(y_train2 == 7, 5, y_train2)
y_train2 = np.where((y_train2 == 13) | (y_train2 == 14), 6, y_train2)
y_train2 = np.where((y_train2 == 15) | (y_train2 == 17)| (y_train2 == 21)| (y_train2 == 25), 7, y_train2)
y_train2 = np.where((y_train2 == 16) | (y_train2 == 18)| (y_train2 == 19)| (y_train2 == 22)| (y_train2 == 23)| (y_train2 == 27)| (y_train2 == 28), 8, y_train2)
y_train2 = np.where(y_train2 == 20, 9, y_train2)
y_train2 = np.where(y_train2 == 24, 10, y_train2)
y_train2 = np.where((y_train2 == 26) | (y_train2 == 29), 11, y_train2)
y_train2 = np.where((y_train2 == 30) | (y_train2 == 32)| (y_train2 == 34)| (y_train2 == 41), 12, y_train2)
y_train2 = np.where((y_train2 == 31) | (y_train2 == 33)| (y_train2 == 35)| (y_train2 == 38)| (y_train2 == 39), 13, y_train2)
y_train2 = np.where((y_train2 == 36) | (y_train2 == 40), 14, y_train2)
y_train2 = np.where(y_train2 == 37, 15, y_train2)
y_train2 = np.where((y_train2 == 42) | (y_train2 == 46)| (y_train2 == 47), 16, y_train2)
y_train2 = np.where((y_train2 == 43) | (y_train2 == 51), 17, y_train2)
y_train2 = np.where((y_train2 == 44) | (y_train2 == 45), 18, y_train2)
y_train2 = np.where(y_train2 == 48 , 19, y_train2)
y_train2 = np.where(y_train2 == 49 , 20, y_train2)
y_train2 = np.where((y_train2 == 50)| (y_train2 == 52), 21, y_train2)
y_train2 = np.where(y_train2 == 53, 22, y_train2)

In [76]:
X_tr2, X_ts2, y_tr2, y_ts2 = train_test_split(X_train, y_train2, random_state = 42, shuffle=True) #снова разбиение
lgbm2 = lgb.LGBMClassifier() #создание объекта классификатора
lgbm2.fit(X_tr2, y_tr2) #обучение модели
y_pr2 = lgbm2.predict(X_ts2) #предсказывание моделью на тестовой выборке
print("%.2f" % accuracy_score(y_ts2, y_pr2)) #вычисление качества метрикой accuracy
                                            #сравнение предсказанных моделью и правильных ответов

0.90


In [77]:
#на данном уровне женская одежда остается одним классом, все остальные объекты остаются в самом детальном виде
y_train3 = data["category_id"]
y_train3 = np.where((y_train3 == 33) | (y_train3 == 35) | (y_train3 == 38) | (y_train3 == 39), 33, y_train3)

In [78]:
X_tr3, X_ts3, y_tr3, y_ts3 = train_test_split(X_train, y_train3, random_state = 42, shuffle=True) #снова разбиение
lgbm3 = lgb.LGBMClassifier() #создание объекта классификатора
lgbm3.fit(X_tr3, y_tr3) #обучение модели
y_pr3 = lgbm3.predict(X_ts3) #предсказывание моделью на тестовой выборке
print("%.2f" % accuracy_score(y_ts3, y_pr3)) #вычисление качества метрикой accuracy
                                            #сравнение предсказанных моделью и правильных ответов

0.86


In [26]:
lgr = LogisticRegression() #создается модель логистической регрессии
lgr.fit(X_tr, y_tr) #обучение 
y_pr = lgr.predict(X_ts) #предсказание  
print(accuracy_score(y_ts, y_pr)) #считаем качество
#в самом начале для проверки использовал как быстрый и простой классификатор, не показал хорошего качества

C:\Users\Bagi\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Bagi\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.19154273574113417


In [4]:
nw_data = text_transform(data["description"][:10]) #первые 10 описаний объявлений

In [5]:
#для дальнейшего улучшения качества, возможно, понадобится сбор некоторых статистик и более детальная обрабокта текста
s = ""
for line in nw_data:
    s += line.strip() + ' '
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
tokenized = word_tokenize(s)

In [6]:
s

'гобелен  размеры 139х84см продам недорого 4 стула из светлой прессованной кожи  стильные  ножки дугообразные мини баня мб 1 мини сауна   предназначена для принятия тепловых процедур в бытовых условиях дома  на даче   а также в спортивных  оздоровительных  косметических  лечебных организациях и учреждениях  она оказывает общеукрепляющее и профилактическое действие на организм человека  номинальное напряжение переменного тока 220 в максимальная мощность 2000 вт максимальная температура нагретого воздуха в термочехле  градусов с  90   100 время достижения рабочей температуры 1 5 мин  время достижения максимальной температуры 10 мин  габаритные размеры тепловой камеры в рабочем состоянии  мм не более    длина 950   ширина 900   высота 1100 масса 6 3   7 3 кг мини баня станет отличным подарком как для мужчин  так и для женщин продам эксклюзивную коллекцию книг  выпущенную ограниченным тиражом по заказу авиакомпании  трансаэро   серия  детективы трансаэро    в коллекцию входит 10 книг    га

In [9]:
l1 = list(map(lambda x: x.lower(), tokenized))
d1 = nltk.FreqDist(l1)
d1.most_common(40) #40 самых популярных слов, конечно, нужно больше текста для понимания 

[('в', 14),
 ('и', 14),
 ('на', 13),
 ('по', 11),
 ('с', 7),
 ('заболевания', 7),
 ('для', 6),
 ('а', 5),
 ('не', 5),
 ('книги', 5),
 ('рублей', 5),
 ('системы', 5),
 ('из', 4),
 ('так', 4),
 ('книг', 4),
 ('авиакомпании', 4),
 ('г', 4),
 ('цена', 4),
 ('от', 4),
 ('4', 3),
 ('мини', 3),
 ('1', 3),
 ('человека', 3),
 ('время', 3),
 ('10', 3),
 ('более', 3),
 ('как', 3),
 ('трансаэро', 3),
 ('о', 3),
 ('12', 3),
 ('до', 3),
 ('часы', 3),
 ('размеры', 2),
 ('продам', 2),
 ('стильные', 2),
 ('баня', 2),
 ('также', 2),
 ('максимальная', 2),
 ('100', 2),
 ('достижения', 2)]